# The Title of The Final Report

Dayna Yoon, Xudong Ma, Evelyn Pu, Shrey Chordia (Group 4)

## **Continue Studying:** ##

## 1. Splitting the Data

In [31]:
heart_data_split <- initial_split(new_heart_data, prop = 0.75, strata = diagnosis)
heart_data_training <- training(heart_data_split)
heart_data_testing <- testing(heart_data_split)

## 2. Which $k$ should we use?

In [ ]:
set.seed(9999) 

# options(repr.plot.height = 5, repr.plot.width = 6)

# k_vals <- tibble(neighbors = seq(from = 2, to = 6))

# recipe1 <- recipe(y ~ .,data = training_set)
# dddd_vfold <- vfold_cv(training_set, v = 5, strata = y)
# knn_spec<- nearest_neighbor(weight_func ="rectangular", neighbors = tune()) |>
#       set_engine("kknn") |>
#       set_mode("classification")
# fit1<- workflow()|>
# add_recipe(recipe1)|>
# add_model(knn_spec)|>
# tune_grid(resamples = dddd_vfold,grid = k_vals )|>
# collect_metrics()

# accuracies<- fit1 |>
# filter(.metric == "accuracy")

# cross_val_plot<-ggplot(accuracies,aes(x=neighbors , y= mean))+
# geom_point()+
# geom_line()+
#  labs(x = "Neighbors", y = "Accuracy Estimate")

# cross_val_plot

knn_recipe<-recipe(diagnosis~age+cholesterol, data = heart_data_training) |>
   step_scale(all_predictors()) |>
   step_center(all_predictors())

knn_model<-nearest_neighbor(weight_func = "rectangular",neighbors = tune())|>
   set_engine("kknn") |>
   set_mode("classification")

knn_vfold <- vfold_cv(heart_data_training,v =10, strata=diagnosis)

grid_vals<-tibble(neighbors=c(1:100))

knn_results <- workflow()|>
      add_recipe(knn_recipe)|>
      add_model(knn_model)